# Image Transformer
1. Image 투명화 
2. 투명 처리된 Image 저장
3. Image 정보(Category, Boundary)를 csv에 저장

In [4]:
import Img_Maker
# 이미지 파일 경로 입력 (메모리 최소 800MB 필요)

# 1 : 이미지 저장
# 0 : 패스
# 2 : 종료

# JSON File Path 설정
FILE_PATH = r'D:\MODEL_DATA\AC\95.json'
Img_Maker.check_img(FILE_PATH = FILE_PATH, save_pos = 0)

마지막 Save Point 33


# 참고자료
https://studyforus.com/innisfree/594134

# 남은 작업
## 학습 데이터 생성기 (이미지 합치기)
1. 매장 이미지(1024x1024)에 투명화한 상품 합치기
2. 매장 이미지 별 BOX 정보 COCO 파일 저장

In [6]:
from PIL import Image
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('image_label.ipynb'))))
from PACK import FileSearch

In [7]:
bg_fl = FileSearch.search('./Emart_Image', '.jpg')

In [9]:
for path in bg_fl:
    temp_bg = Image.open(path)
    x, y = temp_bg.size
    if x < 512 or y < 512:
        os.remove(path)

In [1]:
import Img_Maker
FILE_PATH = r'.\coco_file.csv'

# et_a 는 학습용 이미지 파일이름으로 돌릴때마다 바꿔줘야함 (안그러면 중복됨)
# coco_file.csv 파일 경로, 학습 이미지 파일 이름, 생성할 파일 개수
Img_Maker.mk_TrainData(FILE_PATH, 'et_a', 5)

In [28]:
# 정확한 전처리는 학습시킬 때 확인